# Using [`depfinder`](https://github.com/ericdill/depfinder) to discover the `deathbeds` dependencies.



In [2]:
    from pathlib import Path
    from IPython.utils.capture import capture_output
    from yaml import safe_load
    from collections import defaultdict
    import deathbeds

* Loop all the notebooks in the deathbeds project.

In [14]:
    deps = defaultdict(list)
    for path in map(str, Path(deathbeds.__file__).parent.glob('*.ipynb')): 
        with capture_output() as out:
            !depfinder --yaml --conda $path
        if not out.stdout.strip().startswith('Traceback'):
            current = safe_load(out.stdout.strip())
            if current:
                for key in current: 
                    deps[key] += current[key]

In [26]:
    requirements = set(
        sum(
            (
                list(value) for key, value in deps.items() 
                if key not in ['builtin']), [])
    )

In [30]:
    requirements.add("pandas")

In [36]:
    list(map(requirements.add, {
        'poser', 'importnb', 'depfinder', 'pytest', 'jupyter'
    }));

In [37]:
    Path('../requirements.txt').write_text("\n".join(requirements))

102